In [1]:
import pandas as pd

data = pd.read_csv('data/^NDX_raw_data.csv')
data.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

data_backup = data.iloc[3524:]

data = data.iloc[:3524]
data_copy = data.copy()

print('Data imported and copied.', flush=True)

Data imported and copied.


In [2]:
import numpy as np

## Creating sequences
def create_dataset(dataset, time_step=1, output_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-output_step):
        a = dataset[i:(i+time_step), 0]
        b = dataset[(i+time_step):(i+time_step)+output_step, 0]
        dataX.append(a)
        dataY.append(b)

    return np.array(dataX), np.array(dataY)

period = 60
trend_period = 14
rsi_period = 14
num_features = 1
input_period = 60
output_step = 7
units = 512

In [3]:
print('Initializing the Model for Season...', flush=True)

import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Conv1D, AveragePooling1D, Flatten, Reshape, SimpleRNN, GRU, MaxPooling1D, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.initializers import Zeros

inputs = Input(shape=(input_period, num_features))

model_cnn = Sequential([
    Conv1D(filters=352, kernel_size=1, activation='relu'),
    Conv1D(filters=352, kernel_size=1, activation='relu'),
    MaxPooling1D(pool_size=3),
    Dense(units=128),
    Flatten()
    #Dense(units=output_step),
    #Reshape((output_step,1))
])

model_bilstm = Sequential([
    Bidirectional(LSTM(units=384, return_sequences=True, activation='tanh', recurrent_activation='sigmoid')),
    Dropout(0.2),
    Flatten()
    #Dense(units=output_step),
    #Reshape((output_step,1))
])

model_bigru = Sequential([
    Bidirectional(GRU(units=128, activation='tanh', return_sequences=True)),
    Dropout(0.4),
    Flatten()
])

model_multilayer_lstm = Sequential([
    LSTM(units=64, return_sequences=True, activation='tanh', recurrent_activation='sigmoid'),
    Dropout(0.2),
    LSTM(units=64, return_sequences=True, activation='tanh', recurrent_activation='sigmoid'),
    Flatten()
])

output_cnn = model_cnn(inputs)
output_bilstm = model_bilstm(inputs)
output_bigru = model_bigru(inputs)
output_multilayer_lstm = model_multilayer_lstm(inputs)

concatenated_outputs = concatenate([output_cnn, output_bilstm, output_bigru, output_multilayer_lstm])

main_model = Sequential([
    Input(shape=(concatenated_outputs.shape[1],)),
    Dense(units=output_step),
    Reshape((output_step,1))
])

final_output = main_model(concatenated_outputs)

functional_pipeline_season = Model(inputs=inputs, outputs=final_output)

# Compile the pipeline model
functional_pipeline_season.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

print('Model has been initialized.', flush=True)

Initializing the Model for Season...


2024-06-11 07:06:55.946966: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-11 07:06:56.023468: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 07:06:56.023519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 07:06:56.025997: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-11 07:06:56.039265: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-11 07:06:57.859790: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

Model has been initialized.


In [4]:
print('Initializing the Model for Savitzky-Golay...', flush=True)

import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Conv1D, AveragePooling1D, Flatten, Reshape, SimpleRNN, GRU, MaxPooling1D, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.initializers import Zeros

inputs = Input(shape=(input_period, num_features))

model_cnn = Sequential([
    Conv1D(filters=352, kernel_size=1, activation='relu'),
    Conv1D(filters=352, kernel_size=1, activation='relu'),
    MaxPooling1D(pool_size=3),
    Dense(units=128),
    Flatten()
    #Dense(units=output_step),
    #Reshape((output_step,1))
])

model_bilstm = Sequential([
    Bidirectional(LSTM(units=384, return_sequences=True, activation='tanh', recurrent_activation='sigmoid')),
    Dropout(0.2),
    Flatten()
    #Dense(units=output_step),
    #Reshape((output_step,1))
])

model_bigru = Sequential([
    Bidirectional(GRU(units=128, activation='tanh', return_sequences=True)),
    Dropout(0.4),
    Flatten()
])

model_multilayer_lstm = Sequential([
    LSTM(units=64, return_sequences=True, activation='tanh', recurrent_activation='sigmoid'),
    Dropout(0.2),
    LSTM(units=64, return_sequences=True, activation='tanh', recurrent_activation='sigmoid'),
    Flatten()
])

output_cnn = model_cnn(inputs)
output_bilstm = model_bilstm(inputs)
output_bigru = model_bigru(inputs)
output_multilayer_lstm = model_multilayer_lstm(inputs)

concatenated_outputs = concatenate([output_cnn, output_bilstm, output_bigru, output_multilayer_lstm])

main_model = Sequential([
    Input(shape=(concatenated_outputs.shape[1],)),
    Dense(units=output_step),
    Reshape((output_step,1))
])

final_output = main_model(concatenated_outputs)

functional_pipeline_savgol = Model(inputs=inputs, outputs=final_output)

# Compile the pipeline model
functional_pipeline_savgol.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

print('Model has been initialized.', flush=True)

Initializing the Model for Savitzky-Golay...
Model has been initialized.


In [5]:
best_epoch_backup_savgol = 68

# Load the weights of the model at the chosen epoch
functional_pipeline_savgol.load_weights(f'model_weights_6_5_topological_trend/model_weights_epoch_{best_epoch_backup_savgol:02d}.h5')
print('Backup: Savitzky-Golay Weigths for the best epoch has been loaded.')

Backup: Savitzky-Golay Weigths for the best epoch has been loaded.


In [6]:
best_epoch_backup_season = 30

# Load the weights of the model at the chosen epoch
functional_pipeline_season.load_weights(f'model_weights_6_5_topological/model_weights_epoch_{best_epoch_backup_season:02d}.h5')
print('Backup: Season Weigths for the best epoch has been loaded.')

Backup: Season Weigths for the best epoch has been loaded.


## data testing

In [7]:
data_backup = pd.read_csv('data/^NDX_raw_data.csv')
data_backup.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

data_backup_copy = data_backup.copy()

In [8]:
X_test, y_test = create_dataset(data_backup[['close']].to_numpy(), time_step=period, output_step=output_step)

In [9]:
from statsmodels.tsa.seasonal import seasonal_decompose

decompositions_test = np.array([seasonal_decompose(X_test[i], model='additive', period=14) for i in range(X_test.shape[0])])
seasons_test = np.array([decompositions_test[i].seasonal for i in range(decompositions_test.shape[0])])

In [10]:
from scipy.signal import savgol_filter

# Apply Savitzky-Golay filter
window_length = 17  # Window length (must be odd)
polyorder = 1      # Polynomial order

data_savgol_test = savgol_filter(X_test, window_length, polyorder)

In [11]:
savgol_cropped_test = list()

# NO NEED TO CROP
for _savgol in data_savgol_test:
    savgol_cropped_test.append(_savgol)

savgol_cropped_test = np.array(savgol_cropped_test)

from sklearn.preprocessing import MinMaxScaler

scaler_savgol_test = list(MinMaxScaler() for i in range(savgol_cropped_test.shape[0]))
savgol_scaled_test = list()

for i in range(savgol_cropped_test.shape[0]):
    savgol_scaled_test.append(scaler_savgol_test[i].fit_transform(savgol_cropped_test[i].reshape(-1,1)))

savgol_scaled_test = np.array(savgol_scaled_test)

X_input_savgol_test = list()
#y_input = list()

for savgol in savgol_scaled_test:
    X_input_savgol_test.append(savgol)
    #y_input.append(savgol[-y.shape[1]:])

X_input_savgol_test = np.array(X_input_savgol_test)
#y_input = np.array(y_input)

In [12]:
seasons_cropped_test = list()

for _season in seasons_test:
    seasons_cropped_test.append(_season)

seasons_cropped_test = np.array(seasons_cropped_test)

from sklearn.preprocessing import MinMaxScaler

scaler_seasonal_test = list(MinMaxScaler() for i in range(seasons_cropped_test.shape[0]))
seasons_scaled_test = list()

for i in range(seasons_cropped_test.shape[0]):
    seasons_scaled_test.append(scaler_seasonal_test[i].fit_transform(seasons_cropped_test[i].reshape(-1,1)))

seasons_scaled_test = np.array(seasons_scaled_test)

X_input_season_test = list()
#y_input = list()

for season in seasons_scaled_test:
    X_input_season_test.append(season)
    #y_input.append(lowess[-y.shape[1]:])

X_input_season_test = np.array(X_input_season_test)
#y_input = np.array(y_input)

In [13]:
X_input_savgol_test.shape, X_input_season_test.shape

((4966, 60, 1), (4966, 60, 1))

In [14]:
savgol_predictions_test = list()
season_predictions_test = list()

for j in range(X_input_savgol_test.shape[0]):
    if (j+1) % 100 == 0:
        print(j+1)
        
    savgol_predictions_test.append(
        scaler_savgol_test[j].inverse_transform(
            functional_pipeline_savgol.predict(X_input_savgol_test[j].reshape(savgol_scaled_test[j].shape[1], input_period, num_features), verbose=0)[0].reshape(1,output_step)
        )
    )

    season_predictions_test.append(
        scaler_seasonal_test[j].inverse_transform(
            functional_pipeline_season.predict(X_input_season_test[j].reshape(seasons_scaled_test[j].shape[1], input_period, num_features), verbose=0)[0].reshape(1,output_step)
        )
    )

savgol_predictions_test = np.array(savgol_predictions_test)
season_predictions_test = np.array(season_predictions_test)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [15]:
noise_high, _ = create_dataset(data_backup[['high']].to_numpy(), time_step=period, output_step=output_step)
noise_low, _ = create_dataset(data_backup[['low']].to_numpy(), time_step=period, output_step=output_step)

std_dev_high = np.array([np.std(val) for val in noise_high])
std_dev_low = np.array([np.std(val) for val in noise_low])

std_dev_noise = np.array([np.maximum(val_high, val_low) for val_high, val_low in zip(std_dev_high, std_dev_low)])

In [16]:
observation = pd.Series([
    savgol_predictions_test[i] + 3*season_predictions_test[i] for i in range(savgol_predictions_test.shape[0])
], index=data_backup.index[period:-output_step])

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

pred_reg = list()

scaler_x_reg, scaler_y_reg = MinMaxScaler(), MinMaxScaler()
model_reg = LinearRegression()

for i in observation.index:
    X_reg = np.array([ t for t in range(i, i+output_step) ])
    X_reg_norm = scaler_x_reg.fit_transform(X_reg.reshape(-1,1))

    y_reg = observation[i][0]
    y_reg_norm = scaler_y_reg.fit_transform(y_reg.reshape(-1,1))

    model_reg.fit(X_reg_norm, y_reg_norm)

    pred_reg.append(model_reg.coef_[0][0])

pred_reg = np.array(pred_reg)

In [18]:
pred_reg.shape, observation.index

((4966,), RangeIndex(start=60, stop=5026, step=1))

In [23]:
x_grad = np.array([ t for t in observation.index ])

# Calculate the numerical derivative (dy/dx)
dy_dx = np.gradient(pred_reg, x_grad)

In [34]:
dy_dx

array([ 0.35309029,  0.13837951, -0.06212323, ...,  0.16902556,
        0.41011006,  0.56459958])

In [26]:
# Identify points where the slope changes sign
sign_changes = np.where(np.diff(np.sign(dy_dx)))[0]

In [28]:
sign_changes.shape

(1284,)

In [29]:
# Identify points where the slope changes from negative to positive
neg_to_pos = np.where((np.diff(np.sign(dy_dx)) > 0))[0]

In [33]:
neg_to_pos

array([  10,   19,   23,   31,   45,   52,   58,   66,   76,   82,   89,
         97,  101,  103,  111,  115,  117,  127,  142,  144,  156,  159,
        166,  173,  178,  187,  193,  200,  204,  215,  229,  242,  249,
        256,  263,  271,  284,  292,  295,  301,  308,  315,  321,  335,
        341,  351,  355,  363,  370,  379,  393,  408,  416,  423,  437,
        444,  451,  465,  475,  479,  489,  493,  506,  508,  523,  535,
        544,  548,  558,  562,  574,  585,  594,  601,  615,  621,  628,
        632,  641,  647,  654,  662,  666,  680,  691,  694,  697,  705,
        715,  722,  729,  742,  757,  761,  764,  771,  778,  786,  790,
        805,  812,  824,  830,  840,  842,  845,  854,  860,  874,  880,
        889,  894,  897,  902,  904,  915,  919,  929,  935,  944,  947,
        957,  970,  981,  986,  994,  999, 1007, 1014, 1021, 1027, 1035,
       1042, 1052, 1066, 1073, 1077, 1080, 1088, 1094, 1101, 1104, 1108,
       1116, 1126, 1128, 1133, 1140, 1143, 1148, 11

In [35]:
dy_dx

array([ 0.35309029,  0.13837951, -0.06212323, ...,  0.16902556,
        0.41011006,  0.56459958])

In [42]:
np.sign(dy_dx[:20])

array([ 1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,  1.,
        1.,  1., -1., -1., -1., -1., -1.])

In [43]:
np.diff(np.sign(dy_dx[:20]))

array([ 0., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,
        0., -2.,  0.,  0.,  0.,  0.])

In [45]:
np.where((np.diff(np.sign(dy_dx)) == 2))[0]

array([  10,   19,   23,   31,   45,   52,   58,   66,   76,   82,   89,
         97,  101,  103,  111,  115,  117,  127,  142,  144,  156,  159,
        166,  173,  178,  187,  193,  200,  204,  215,  229,  242,  249,
        256,  263,  271,  284,  292,  295,  301,  308,  315,  321,  335,
        341,  351,  355,  363,  370,  379,  393,  408,  416,  423,  437,
        444,  451,  465,  475,  479,  489,  493,  506,  508,  523,  535,
        544,  548,  558,  562,  574,  585,  594,  601,  615,  621,  628,
        632,  641,  647,  654,  662,  666,  680,  691,  694,  697,  705,
        715,  722,  729,  742,  757,  761,  764,  771,  778,  786,  790,
        805,  812,  824,  830,  840,  842,  845,  854,  860,  874,  880,
        889,  894,  897,  902,  904,  915,  919,  929,  935,  944,  947,
        957,  970,  981,  986,  994,  999, 1007, 1014, 1021, 1027, 1035,
       1042, 1052, 1066, 1073, 1077, 1080, 1088, 1094, 1101, 1104, 1108,
       1116, 1126, 1128, 1133, 1140, 1143, 1148, 11